## 

In [13]:
# Import dependencies

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import sqlite3
from flask import Flask, jsonify
import pandas as pd

# Set path Geckodriver executable
geckodriver_path = "/usr/local/bin/geckodriver"  

# Configure Firefox options
firefox_options = Options()
firefox_options.headless = True  

# Set Firefox web driver
firefox_service = FirefoxService(geckodriver_path)
driver = webdriver.Firefox(service=firefox_service, options=firefox_options)

# Url scrape
url = "https://steamdb.info/charts/"

# Load the page
driver.get(url)

# Add a delay of 3 seconds to allow the table to initialize
time.sleep(3)

wait = WebDriverWait(driver, 30)  # Increased the wait time to 30 seconds

# Wait for the table to become visible using XPath and wait for "Initializing table" message to disappear
try:
    wait.until_not(EC.visibility_of_element_located((By.XPATH, "//div[text()='Initializing table']")))
except TimeoutException:
    print("Table initialization timed out.")
    driver.quit()

data_list = []

# Scrape data from the first sheet (button "1" is already selected by default)
table = driver.find_element(By.CSS_SELECTOR, "table.table-products")
rows = table.find_elements(By.XPATH, "//table[@id='table-apps']/tbody/tr")
for row in rows:
    columns = row.find_elements(By.TAG_NAME, "td")
    number = columns[0].text.strip()
    name = columns[2].text.strip()
    current = columns[3].text.strip()
    peak_24h = columns[4].text.strip()
    peak_all_time = columns[5].text.strip()

    game_data = {
        "Number": number,
        "Name": name,
        "Current": current,
        "24h Peak": peak_24h,
        "All-Time Peak": peak_all_time,
    }

    data_list.append(game_data)

# Click the "2" link to switch to the second sheet
try:
    link_2 = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, '2')))
    link_2.click()
except TimeoutException:
    print("Link '2' not found or clickable.")
    driver.quit()

# Add a delay to allow the second sheet to load
time.sleep(3)  # Adjust the delay as needed

# Scrape data from the second sheet
table = driver.find_element(By.CSS_SELECTOR, "table.table-products")
rows = table.find_elements(By.XPATH, "//table[@id='table-apps']/tbody/tr")
for row in rows:
    columns = row.find_elements(By.TAG_NAME, "td")
    number = columns[0].text.strip()
    name = columns[2].text.strip()
    current = columns[3].text.strip()
    peak_24h = columns[4].text.strip()
    peak_all_time = columns[5].text.strip()

    game_data = {
        "Number": number,
        "Name": name,
        "Current": current,
        "24h Peak": peak_24h,
        "All-Time Peak": peak_all_time,
    }

    data_list.append(game_data)

# Close the browser
driver.quit()

# Now, data_list contains the scraped data from both sheets
for game in data_list:
    print(game)


{'Number': '1.', 'Name': 'Counter-Strike: Global Offensive', 'Current': '776,003', '24h Peak': '1,285,148', 'All-Time Peak': '1,818,773'}
{'Number': '2.', 'Name': 'Dota 2', 'Current': '265,616', '24h Peak': '651,740', 'All-Time Peak': '1,295,114'}
{'Number': '3.', 'Name': "Baldur's Gate 3", 'Current': '154,178', '24h Peak': '284,503', 'All-Time Peak': '875,343'}
{'Number': '4.', 'Name': 'Cyberpunk 2077', 'Current': '148,685', '24h Peak': '167,465', 'All-Time Peak': '1,054,388'}
{'Number': '5.', 'Name': 'Apex Legends', 'Current': '141,639', '24h Peak': '397,605', 'All-Time Peak': '624,473'}
{'Number': '6.', 'Name': 'PUBG: BATTLEGROUNDS', 'Current': '139,004', '24h Peak': '382,200', 'All-Time Peak': '3,257,248'}
{'Number': '7.', 'Name': 'NARAKA: BLADEPOINT', 'Current': '79,378', '24h Peak': '217,063', 'All-Time Peak': '264,406'}
{'Number': '8.', 'Name': 'Starfield', 'Current': '67,016', '24h Peak': '129,187', 'All-Time Peak': '330,723'}
{'Number': '9.', 'Name': 'Wallpaper Engine', 'Curre

In [14]:
# SQLite Database

# Create a connection to the SQLite database 
conn = sqlite3.connect("steam_data.db")

# Create a cursor object
cursor = conn.cursor()

In [15]:
# Define the table schema
cursor.execute('''
    CREATE TABLE IF NOT EXISTS games (
        Number TEXT,
        Name TEXT,
        Current TEXT,
        Peak_24h TEXT,
        Peak_All_Time TEXT
    )
''')

In [16]:
# Insert data into the table
for game in data_list:
    cursor.execute('''
        INSERT INTO games (Number, Name, Current, Peak_24h, Peak_All_Time)
        VALUES (?, ?, ?, ?, ?)
    ''', (game["Number"], game["Name"], game["Current"], game["24h Peak"], game["All-Time Peak"]))

In [17]:
# Commit changes
conn.commit()
cursor.close()
conn.close()


In [18]:
# Create a DataFrame from the SQLite database
conn = sqlite3.connect("steam_data.db")
df = pd.read_sql_query("SELECT * FROM games", conn)
conn.close()
df.tail()

,Number,Name,Current,Peak_24h,Peak_All_Time
2795,196.,Fallout: New Vegas,"2,413","3,920","51,038"
2796,197.,Yu-Gi-Oh! Duel Links,"2,413","3,439","21,520"
2797,198.,Overcooked! 2,"2,399","8,365","12,108"
2798,199.,大侠立志传：碧血丹心,"2,379","4,533","28,936"
2799,200.,Grounded,"2,355","4,933","32,012"


In [11]:

# Connect to the SQLite database
conn = sqlite3.connect("steam_data.db")
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='games';")

# Fetch the result
result = cursor.fetchone()

# Check if the result is not None (table exists)
if result:
    print("The 'games' table exists in the database.")
else:
    print("The 'games' table does not exist in the database.")

# Close the cursor (optional, but recommended)
cursor.close()

# Close the database connection
conn.close()



The 'games' table exists in the database.
